***

# Проектная работа: Рынок заведений общественного питания Москвы

## Загрузим данные и изучим общую информацию

In [1]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.express as px
import json
from folium import Map, Choropleth
from folium import Marker, Map
from folium.plugins import MarkerCluster

ModuleNotFoundError: No module named 'folium'

In [ ]:
try:
    data = pd.read_csv('/datasets/moscow_places.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/moscow_places.csv')

In [ ]:
data.head()

In [ ]:
data.info()

**Вывод:**
- Датасет содержит 8406 строк
- Названия столбцов в едином стиле
- Типы столбцов корректные
- Есть пропуски в столбцах *hours, price, avg_bill, middle_coffee_cup* и *seats* 

## Выполним предобработку данных

### Изучим, есть ли дубликаты и найдем пропуски

Проверим есть ли дубликаты в датасете

In [ ]:
data.duplicated().sum()

Дубликатов нет

Изучим столбец *name*

In [ ]:
data['name'].describe()

In [ ]:
data['name'].duplicated().sum()

2792 дубликата. Удалять их не будем, так как это могут быть сетевые заведения

Изучим столбец *category*

In [ ]:
data['category'].unique()

Здесь все в порядке

Изучим столбец *address*

In [ ]:
data['address'].duplicated().sum()

2653 дубликата. Так же оставим как есть. Несколько заведений могут находиться по одному адресу. Например в торговых центрах

Посмотрим есть ли неявные дубликаты. Для этого приведем столбцы *name* и *address* к нижнему регистру

In [ ]:
data['name'] = data['name'].str.lower()
data['address'] = data['address'].str.lower()

In [ ]:
data[['name', 'address']].duplicated().sum()

4 неявных дубликата

In [ ]:
data[data[['name', 'address']].duplicated(keep=False)]

Удалим их

In [ ]:
data = data.drop_duplicates(subset=['name', 'address'], keep='last')
data[['name', 'address']].duplicated().sum()

Изучим столбец *district*

In [ ]:
data['district'].unique()

С округами все в порядке

Пропуски в столбцах оставим как есть. Корректно заполнить их мы не сможем, а удаление такого количества строк может повлиять на исследование

### Создадим новые столбцы

Создадим столбец *street* с названиями улиц из столбца с адресом

In [ ]:
data['street'] = data['address'].str.split(',').str[1].str.strip()

Создадим столбец is_24/7 с обозначением, что заведение работает ежедневно и круглосуточно (24/7):
- логическое значение True — если заведение работает ежедневно и круглосуточно;
- логическое значение False — в противоположном случае.

In [ ]:
data['is_24/7'] = data['hours'].fillna('').str.contains('ежедневно, круглосуточно')

In [ ]:
data.head()

**Вывод:**
- Провели предобработку данных
- Создали новые столбцы *street* и *is_24/7*

## Анализ данных

### Исследуем количество объектов общественного питания по категориям

In [ ]:
category = data.groupby('category').agg(count=pd.NamedAgg(column="category", aggfunc="count")).sort_values(by='count', ascending=False).reset_index()
category

In [ ]:
plt.figure(figsize=(18, 10))
ax = sns.barplot(x='category', y='count', data=category, color = 'seagreen')
ax.set_title('Количество объектов общественного питания по категориям')


In [ ]:
category['percentage'] = round(category['count'] / category['count'].sum() * 100)
category

Больше всех кафе, меньше всех булочных

### Исследуем количество посадочных мест в местах по категориям

In [ ]:
category_seats = data.groupby('category').agg(median=pd.NamedAgg(column="seats", aggfunc="median")).sort_values(by='median', ascending=False).reset_index()
category_seats

In [ ]:
plt.figure(figsize=(18, 10))
ax = sns.barplot(x='category', y='median', data=category_seats, color='seagreen')
ax.set_title('Количество посадочных мест в местах по категориям')
ax.set_xlabel('Категория заведения')
ax.set_ylabel('Количество посадочных мест')

В среднем в ресторанах больше всего посадочных мест, меньше всего - в булочных 

### Рассмотрим и изобразим соотношение сетевых и несетевых заведений в датасете

In [ ]:
chain = data.groupby('chain').agg(count=pd.NamedAgg(column="chain", aggfunc="count")).reset_index()
chain

In [ ]:
plt.figure(figsize=(18, 10))
colors = sns.color_palette('pastel')[ 0:5 ]
labels = ['Сетевые заведения', 'Несетевые заведения']
plt.title('Cоотношение сетевых и несетевых заведений')
plt.pie(chain['count'], labels = labels, colors = colors, autopct='%.0f%%')
plt.show()

Сетевых заведений больше, чем несетевых

### Исследуем какие категории заведений чаще являются сетевыми

In [ ]:
category_chain = data.groupby('category').agg(count=pd.NamedAgg(column="category", aggfunc="count"), chain=pd.NamedAgg(column="chain", aggfunc="sum"))
category_chain['chain_%'] = round(category_chain['chain'] / category_chain['count'] * 100)
category_chain = category_chain.sort_values(by='chain_%', ascending=False).reset_index()
category_chain

In [ ]:
plt.figure(figsize=(18, 10))
ax = sns.barplot(x='category', y='chain_%', data=category_chain, color='seagreen')
ax.set_title('% сетевых заведений по категориям')
ax.set_xlabel('% сетевых заведений')
ax.set_ylabel('Количество посадочных мест')

Чаще всего сетевыми являются булочные, реже всего - бар, паб

### Найдем топ-15 популярных сетей в Москве. 

In [ ]:
name = data.groupby('name').agg(count=pd.NamedAgg(column="name", aggfunc="count")).sort_values(by='count', ascending=False).reset_index()
name = name.head(15)
name

In [ ]:
plt.figure(figsize=(18, 16))
ax = sns.barplot(x='count', y='name', data=name)
ax.set_title('Топ-15 популярных сетей в Москве')
ax.set_xlabel('Количество заведений')
ax.set_ylabel('Название сети')

Самое популярное заведение - кафе

### Исследуем какие административные районы Москвы присутствуют в датасете и отобразим количество заведений

In [ ]:
data['district'].value_counts()

In [ ]:
district = pd.pivot_table(data, values='name', index=["category"], columns =["district"], aggfunc='count').reset_index()

district

In [ ]:
district = district.sort_values(by='Восточный административный округ', ascending=False)
plt.figure(figsize=(18, 10))
ax = sns.barplot(x='category', y='Восточный административный округ', data=district, color='seagreen')
ax.set_title('Количество заведений Восточный административный округ')
ax.set_xlabel('Категория заведения')
ax.set_ylabel('Количество заведений')

district = district.sort_values(by='Западный административный округ', ascending=False)
plt.figure(figsize=(18, 10))
ax = sns.barplot(x='category', y='Западный административный округ', data=district, color='seagreen')
ax.set_title('Количество заведений Западный административный округ')
ax.set_xlabel('Категория заведения')
ax.set_ylabel('Количество заведений')

district = district.sort_values(by='Северный административный округ', ascending=False)
plt.figure(figsize=(18, 10))
ax = sns.barplot(x='category', y='Северный административный округ', data=district, color='seagreen')
ax.set_title('Количество заведений Северный административный округ')
ax.set_xlabel('Категория заведения')
ax.set_ylabel('Количество заведений')

district = district.sort_values(by='Северо-Восточный административный округ', ascending=False)
plt.figure(figsize=(18, 10))
ax = sns.barplot(x='category', y='Северо-Восточный административный округ', data=district, color='seagreen')
ax.set_title('Количество заведений Северо-Восточный административный округ')
ax.set_xlabel('Категория заведения')
ax.set_ylabel('Количество заведений')

district = district.sort_values(by='Северо-Западный административный округ', ascending=False)
plt.figure(figsize=(18, 10))
ax = sns.barplot(x='category', y='Северо-Западный административный округ', data=district, color='seagreen')
ax.set_title('Количество заведений Северо-Западный административный округ')
ax.set_xlabel('Категория заведения')
ax.set_ylabel('Количество заведений')

district = district.sort_values(by='Центральный административный округ', ascending=False)
plt.figure(figsize=(18, 10))
ax = sns.barplot(x='category', y='Центральный административный округ', data=district, color='seagreen')
ax.set_title('Количество заведений Центральный административный округ')
ax.set_xlabel('Категория заведения')
ax.set_ylabel('Количество заведений')

district = district.sort_values(by='Юго-Восточный административный округ', ascending=False)
plt.figure(figsize=(18, 10))
ax = sns.barplot(x='category', y='Юго-Восточный административный округ', data=district, color='seagreen')
ax.set_title('Количество заведений Юго-Восточный административный округ')
ax.set_xlabel('Категория заведения')
ax.set_ylabel('Количество заведений')

district = district.sort_values(by='Юго-Западный административный округ', ascending=False)
plt.figure(figsize=(18, 10))
ax = sns.barplot(x='category', y='Юго-Западный административный округ', data=district, color='seagreen')
ax.set_title('Количество заведений Юго-Западный административный округ')
ax.set_xlabel('Категория заведения')
ax.set_ylabel('Количество заведений')

district = district.sort_values(by='Южный административный округ', ascending=False)
plt.figure(figsize=(18, 10))
ax = sns.barplot(x='category', y='Южный административный округ', data=district, color='seagreen')
ax.set_title('Количество заведений Южный административный округ')
ax.set_xlabel('Категория заведения')
ax.set_ylabel('Количество заведений')

Больше всего заведений в Центральном административном округе. Во всех округах больше всего кафе и ресторанов

### Визуализируем распределение средних рейтингов по категориям заведений

In [ ]:
rating = data.groupby('category').agg(rating=pd.NamedAgg(column="rating", aggfunc="mean")).sort_values(by='rating', ascending=False).reset_index()
rating['rating'] = round(rating['rating'], 2)
rating

In [ ]:
round(rating['rating'].mean(), 1)

In [ ]:
plt.figure(figsize=(18, 10))
ax = sns.barplot(x='category', y='rating', data=rating, color='blueviolet')
ax.set_title('Cредний рейтингов по категориям заведений')
ax.set_xlabel('Категория заведения')
ax.set_ylabel('Рейтинг')

Рейтинги практически не отличаются

### Построим фоновую картограмму (хороплет) со средним рейтингом заведений каждого района

In [ ]:
rating_district = data.groupby('district').agg(rating=pd.NamedAgg(column="rating", aggfunc="mean")).sort_values(by='rating', ascending=False).reset_index()
rating_district['rating'] = round(rating_district['rating'], 2)
rating_district

In [ ]:
round(rating_district['rating'].mean(), 1)

In [ ]:
state_geo = '/datasets/admin_level_geomap.geojson'
moscow_lat, moscow_lng = 55.751244, 37.618423
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
Choropleth(
    geo_data=state_geo,
    data=rating_district,
    columns=['district', 'rating'],
    key_on='feature.name',
    fill_color='YlGn',
    fill_opacity=0.8,
    legend_name='Медианный рейтинг заведений по районам',
).add_to(m)
m

В Центральном административном округе рейтинги чуть выше

### Отобразим все заведения датасета на карте с помощью кластеров средствами библиотеки folium.

In [ ]:
moscow_lat, moscow_lng = 55.751244, 37.618423
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
marker_cluster = MarkerCluster().add_to(m)

def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)

data.apply(create_clusters, axis=1)
m

На картограмме также видно, что в Центральном округе больше всего заведений

### Найдем топ-15 улиц по количеству заведений

In [ ]:
street = data.groupby('street').agg(count=pd.NamedAgg(column="name", aggfunc="count")).sort_values(by='count', ascending=False).reset_index()
street_15 = street.head(15)
street_15

In [ ]:
plt.figure(figsize=(18, 16))
ax = sns.barplot(x='count', y='street', data=street_15)
ax.set_title('Топ-15 улиц по количеству заведений')
ax.set_xlabel('количество заведений')
ax.set_ylabel('Название улицы')

Больше всего заведений на Проспекте мира

### Найдем улицы, на которых находится только один объект общепита

In [ ]:
street[street['count'] == 1].count()

458 улиц с одним объектом общепита

### Посчитаем средний чек для каждого  района.

In [ ]:
middle_avg_bill = data.groupby('district').agg(median=pd.NamedAgg(column="middle_avg_bill", aggfunc="median")).sort_values(by='median', ascending=False).reset_index()
middle_avg_bill

In [ ]:
state_geo = '/datasets/admin_level_geomap.geojson'
moscow_lat, moscow_lng = 55.751244, 37.618423
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
Choropleth(
    geo_data=state_geo,
    data=middle_avg_bill,
    columns=['district', 'median'],
    key_on='feature.name',
    fill_color='YlGn',
    fill_opacity=0.8,
    legend_name='Медианная стоимость заказа по районам',
).add_to(m)
m

Самый большой средний чек в Центральном и Западном округах

**Вывод:**
- Больше всего заведений в категории кафе - 2378 или 28%
- Посадочных мест больше всего в ресторанах - в среднем 86
- Чаще заведения являются сетевыми - 62%
- Самая популярная сеть - Кафе
- Больше всего заведений в Центральном административном округе - 2242
- Средний рейтинг по категориям не сильно отличается и составляет - 4.3
- По районам рейтинг также примерно равен
- Больше всего заведений на проспекте Мира - 184
- Улиц с одним объектом общепита - 458
- Самый большой средний чек в Центральном и Западном округах - 1000

## Детализируем исследование: открытие кофейни

### Посмотрим сколько всего кофеен в датасете

In [ ]:
category[category['category'] == 'кофейня']

1413 кофеен

In [ ]:
coffee_house_dist = district[district['category'] == 'кофейня']
coffee_house_dist

Больше всего кофеен в Центральном административном округе - 428

In [ ]:
coffee_house = data[data['category'] == 'кофейня']

moscow_lat, moscow_lng = 55.751244, 37.618423
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
marker_cluster = MarkerCluster().add_to(m)

def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)

coffee_house.apply(create_clusters, axis=1)
m

### Посмотрим есть ли круглосуточные кофейни

In [ ]:
coffee_house[coffee_house['is_24/7'] == True].info()

59 круглосуточных кофеен

### Посмотрим какие у кофеен рейтинги, как они распределяются по районам

In [ ]:
rating_coffee_house = coffee_house.groupby('district').agg(rating=pd.NamedAgg(column="rating", aggfunc="mean")).sort_values(by='rating', ascending=False).reset_index()
rating_coffee_house['rating'] = round(rating_coffee_house['rating'], 2)
rating_coffee_house

In [ ]:
plt.figure(figsize=(18, 10))
ax = sns.barplot(x='district', y='rating', data=rating_coffee_house, color='blueviolet')
ax.set_title('Cредний рейтинг кофеен по районам')
ax.set_xlabel('Округ')
ax.set_ylabel('Рейтинг')
plt.xticks(rotation=10)

Рейтинги такие же как и в других заведениях

### Посмотрим на какую стоимость чашки капучино стоит ориентироваться при открытии

In [ ]:
middle_coffee_cup = coffee_house.groupby('district').agg(middle_coffee_cup=pd.NamedAgg(column="middle_coffee_cup", aggfunc="median")).sort_values(by='middle_coffee_cup', ascending=False).reset_index()
middle_coffee_cup

Стоимость зависит от района

**Вывод:**

Заказчики не боятся конкуренции, значит стоит рассмотреть открытие кофейни в Центральном административном округе. Это центр, там больше всего заведений, а значит и людей. Так же стоит обратить внимание на Западный административный округ. Средняя оценка заведений там, чуть ниже, чем в других округах. Это значит, что крутых кофеен там не хватает. Средняя стоимость чашки капучино, такая же как и в центре. Не рекомендуется открывать круглосуточную кафейню. Таких практически нет, а значит в этом нет необходимости.

Презентация: https://disk.yandex.ru/i/7s5sF9jmBNKA8w

***